# 🖥 Synthesizing Tool Use Training Data with Gretel Navigator for Computer Control

This notebook demonstrates how to synthesize high-quality datasets for function-calling scenarios involving computer control. Using the JSON Schema standard, we generate realistic function invocations, natural language commands, and rejection scenarios.

While this example is built around tool use data for controlling a personal computer, it can be easily adapted to any tool use scenario following the **JSON Schema standard**.

The resulting datasets are designed to fine-tune AI models or enable in-context learning for Retrieval-Augmented Generation (RAG) workflows.

## 🔍 What We'll Do:

- Define computer control functions, such as text editing, shell commands, and web automation.
- Generate diverse invocations with valid parameter combinations.
- Create natural language commands corresponding to the invocations.
- Synthesize rejection commands for unsupported tasks.
- Analyze dataset coverage to ensure diversity and balance.

## 🚀 Why It Matters:

These datasets enable AI systems to:
- Accurately perform tasks like opening files, editing text, or running scripts.
- Safely reject ambiguous or unsupported requests.
- Improve reliability and safety in real-world applications.

This workflow is purpose-built for training AI models to interact with computer environments. Whether for fine-tuning or in-context learning, these datasets empower systems to perform precise and reliable tool use.

Let’s get started! 🎯

In [ ]:
# Setup and Dependencies

# Install requirements (uncomment if running in a Colab environment)
!pip install -qq gretel-client pandas rich

# Gretel API
from gretel_client import Gretel

# Data Manipulation
import pandas as pd
import json
from collections import Counter

# Prompt Formatting
from textwrap import dedent

# Utilities
import itertools
from typing import Dict, List, Any, Generator
from itertools import product, combinations
from pprint import pprint

# Visualization
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.syntax import Syntax
from rich.text import Text


In [ ]:
# Define computer control functions and their parameters

# These functions simulate real-world computer interactions, enabling precise AI-driven function calls.
# They will be used to generate test data for synthesizing and evaluating AI reasoning capabilities.

COMPUTER_SYSTEM_PROMPT = """
You are an intelligent assistant capable of controlling a computer system. Based on user instructions, you can perform a wide range of tasks such as interacting with files, managing system resources, executing commands, and web-based operations.

Your task is to evaluate the user's request and decide whether it can be completed using the available functions. If possible, call the most appropriate function(s) with valid parameters to achieve the goal. If the request is ambiguous or unsupported, use the `reject_request` function.

### Guidelines:
1. Ensure that the selected function and its parameters match the user's intent.
2. Validate that all required parameters are included and meet their constraints.
3. If the user's request involves multiple steps, break it into sequential function calls where possible.
4. Reject requests that:
   - Are not supported by the available functions.
   - Have missing or ambiguous details.
   - Pose risks or require permissions beyond your scope (e.g., accessing sensitive data).
"""

function_list = [
    # General Computer Control
    {
        "type": "function",
        "function": {
            "name": "open_application",
            "description": "Open an application by its name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "application_name": {
                        "type": "string",
                        "description": "The name of the application to open (e.g., 'Email', 'Chrome')."
                    }
                },
                "required": ["application_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "close_application",
            "description": "Close an application by its name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "application_name": {
                        "type": "string",
                        "description": "The name of the application to close."
                    }
                },
                "required": ["application_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "type_text",
            "description": "Simulate typing text into the active window or field.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "The text to type."
                    }
                },
                "required": ["text"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "press_key",
            "description": "Simulate pressing a key or combination of keys.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keys": {
                        "type": "string",
                        "description": "The key or sequence of keys to press (e.g., 'Ctrl+C', 'Alt+Tab')."
                    }
                },
                "required": ["keys"],
            },
        },
    },

    # File Management
    {
        "type": "function",
        "function": {
            "name": "open_file",
            "description": "Open a file by name or path.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to open (e.g., 'report.docx')."
                    },
                    "file_path": {
                        "type": "string",
                        "description": "The absolute or relative path of the file to open."
                    }
                },
                "required": [],
                "oneOf": [
                    {"required": ["file_name"]},
                    {"required": ["file_path"]}
                ],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_file",
            "description": "Delete a file by name or path.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to delete."
                    },
                    "file_path": {
                        "type": "string",
                        "description": "The absolute or relative path of the file to delete."
                    }
                },
                "required": [],
                "oneOf": [
                    {"required": ["file_name"]},
                    {"required": ["file_path"]}
                ],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "move_file",
            "description": "Move a file to a new location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to move."
                    },
                    "destination": {
                        "type": "string",
                        "description": "The destination location (e.g., 'D drive', 'Desktop')."
                    },
                    "source_path": {
                        "type": "string",
                        "description": "The current path of the file, if known."
                    }
                },
                "required": ["file_name", "destination"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "copy_file",
            "description": "Copy a file to a new location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to copy."
                    },
                    "destination": {
                        "type": "string",
                        "description": "The destination location (e.g., 'Documents folder')."
                    },
                    "source_path": {
                        "type": "string",
                        "description": "The current path of the file, if known."
                    }
                },
                "required": ["file_name", "destination"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "rename_file",
            "description": "Rename a file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "current_name": {
                        "type": "string",
                        "description": "The current name of the file."
                    },
                    "new_name": {
                        "type": "string",
                        "description": "The new name for the file."
                    },
                    "file_path": {
                        "type": "string",
                        "description": "The path of the file, if known."
                    }
                },
                "required": ["current_name", "new_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "find_file",
            "description": "Search for a file by name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to find."
                    },
                    "search_locations": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "description": "Locations to search (e.g., 'C drive', 'Documents folder')."
                        },
                        "description": "Optional list of locations to search in."
                    }
                },
                "required": ["file_name"],
            },
        },
    },

    # Web and Text Interaction
    {
        "type": "function",
        "function": {
            "name": "open_website",
            "description": "Open a website in the default browser.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The web address to open (e.g., 'https://www.example.com')."
                    }
                },
                "required": ["url"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Perform a web search using the default browser.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search term or query."
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "find_and_replace",
            "description": "Find and replace text within a file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The name of the file to modify."
                    },
                    "search_text": {
                        "type": "string",
                        "description": "The text to find."
                    },
                    "replace_text": {
                        "type": "string",
                        "description": "The text to replace with."
                    },
                    "file_path": {
                        "type": "string",
                        "description": "The path to the file, if known."
                    }
                },
                "required": ["file_name", "search_text", "replace_text"],
            },
        },
    },

    # System Control
    {
        "type": "function",
        "function": {
            "name": "adjust_volume",
            "description": "Set the system volume level.",
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "description": "The desired volume level (0-100).",
                        "minimum": 0,
                        "maximum": 100
                    }
                },
                "required": ["level"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "mute_volume",
            "description": "Mute or unmute the system volume.",
            "parameters": {
                "type": "object",
                "properties": {
                    "mute": {
                        "type": "boolean",
                        "description": "True to mute, false to unmute."
                    }
                },
                "required": ["mute"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "shutdown_system",
            "description": "Shut down the computer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "delay_seconds": {
                        "type": "integer",
                        "description": "Time in seconds before shutdown. Default is immediate.",
                        "minimum": 0
                    }
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "restart_system",
            "description": "Restart the computer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "delay_seconds": {
                        "type": "integer",
                        "description": "Time in seconds before restart. Default is immediate.",
                        "minimum": 0
                    }
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "lock_system",
            "description": "Lock the computer screen.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },

    # Navigation and Interaction
    {
        "type": "function",
        "function": {
            "name": "click_element",
            "description": "Click on a UI element identified by text or accessibility label.",
            "parameters": {
                "type": "object",
                "properties": {
                    "element_identifier": {
                        "type": "string",
                        "description": "The text or label identifying the UI element to click."
                    }
                },
                "required": ["element_identifier"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "scroll_page",
            "description": "Scroll the active window or page.",
            "parameters": {
                "type": "object",
                "properties": {
                    "direction": {
                        "type": "string",
                        "enum": ["up", "down", "left", "right"],
                        "description": "The direction to scroll."
                    },
                    "amount": {
                        "type": "integer",
                        "description": "The amount to scroll. Default is a standard increment.",
                        "minimum": 1
                    }
                },
                "required": ["direction"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "navigate_to",
            "description": "Open a folder or directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "folder_name": {
                        "type": "string",
                        "description": "The name of the folder to open (e.g., 'Documents', 'D drive')."
                    },
                    "folder_path": {
                        "type": "string",
                        "description": "The path of the folder to open, if known."
                    }
                },
                "required": [],
                "oneOf": [
                    {"required": ["folder_name"]},
                    {"required": ["folder_path"]}
                ],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_files",
            "description": "Search for files or folders on the computer.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search term."
                    },
                    "location": {
                        "type": "string",
                        "description": "The directory to search within, if any."
                    }
                },
                "required": ["query"],
            },
        },
    },

    # Communication and Media
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Compose and send an email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "recipient": {
                        "type": "string",
                        "description": "Email address of the recipient."
                    },
                    "subject": {
                        "type": "string",
                        "description": "Subject line of the email."
                    },
                    "body": {
                        "type": "string",
                        "description": "Body content of the email."
                    }
                },
                "required": ["recipient", "subject", "body"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "play_media",
            "description": "Play a media file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "media_name": {
                        "type": "string",
                        "description": "The name of the media file to play."
                    },
                    "media_path": {
                        "type": "string",
                        "description": "The path to the media file, if known."
                    }
                },
                "required": [],
                "oneOf": [
                    {"required": ["media_name"]},
                    {"required": ["media_path"]}
                ],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "pause_media",
            "description": "Pause the currently playing media.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },

    # System Information and Settings
    {
        "type": "function",
        "function": {
            "name": "check_system_status",
            "description": "Retrieve current system status information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "status_type": {
                        "type": "string",
                        "enum": ["battery", "network", "cpu_usage", "memory_usage"],
                        "description": "Type of status information to retrieve."
                    }
                },
                "required": ["status_type"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_system",
            "description": "Check for and install system updates.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "change_setting",
            "description": "Change a system setting.",
            "parameters": {
                "type": "object",
                "properties": {
                    "setting_name": {
                        "type": "string",
                        "description": "The name of the setting to change."
                    },
                    "value": {
                        "type": ["string", "integer", "boolean"],
                        "description": "The new value for the setting."
                    }
                },
                "required": ["setting_name", "value"],
            },
        },
    },

    # Assistance and Accessibility
    {
        "type": "function",
        "function": {
            "name": "open_help",
            "description": "Open the help documentation or support page.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "Specific help topic to open."
                    }
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "start_screen_reader",
            "description": "Activate the screen reader for accessibility.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },

    # Advanced Operations
    {
        "type": "function",
        "function": {
            "name": "execute_script",
            "description": "Execute a script file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "script_path": {
                        "type": "string",
                        "description": "The absolute path to the script file."
                    }
                },
                "required": ["script_path"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "manage_process",
            "description": "Start, stop, or restart a process.",
            "parameters": {
                "type": "object",
                "properties": {
                    "process_name": {
                        "type": "string",
                        "description": "The name of the process."
                    },
                    "action": {
                        "type": "string",
                        "enum": ["start", "stop", "restart"],
                        "description": "Action to perform on the process."
                    }
                },
                "required": ["process_name", "action"],
            },
        },
    },

    # Rejecting Requests
    {
        "type": "function",
        "function": {
            "name": "reject_request",
            "description": "Reject unsupported, ambiguous, or risky requests.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "Explanation of why the request was rejected."
                    }
                },
                "required": ["reason"],
            },
        },
    },
]


In [ ]:
# Helper functions for generating permutations of parameters


def get_possible_values(params: Dict[str, Dict[str, Any]], field: str) -> List[Any]:
    """
    Get all possible values for a given field based on its type and constraints.
    """
    field_info = params["properties"][field]

    if "enum" in field_info:
        return field_info["enum"]
    elif field_info["type"] == "integer":
        return ["fill_in_int"]  # Placeholder for integers
    elif field_info["type"] == "string":
        return ["fill_in_string"]  # Placeholder for strings
    elif field_info["type"] == "boolean":
        return [True, False]
    elif field_info["type"] == "object":
        return ["fill_in_object"]  # Placeholder for complex objects
    return []

def generate_parameter_permutations(params: Dict[str, Dict[str, Any]], required_fields: List[str]) -> List[Dict[str, Any]]:
    """
    Generate all possible permutations of required parameters.
    """
    required_values = [get_possible_values(params, field) for field in required_fields]
    return [dict(zip(required_fields, values))
            for values in itertools.product(*required_values)]

def generate_all_permutations(params: Dict[str, Dict[str, Any]]) -> Generator[Dict[str, Any], None, None]:
    """
    Generate all possible permutations including optional parameters.

    Args:
        params: Parameter dictionary containing field definitions

    Returns:
        Generator yielding dictionaries of parameter permutations
    """
    required_fields = params.get("required", [])
    required_perms = generate_parameter_permutations(params, required_fields)

    optional_fields = list(set(params["properties"]) - set(required_fields))

    for base_perm in required_perms:
        yield base_perm

        # Generate combinations of optional fields
        for r in range(1, len(optional_fields) + 1):
            for optional_combo in itertools.combinations(optional_fields, r):
                optional_values = [get_possible_values(params, field) for field in optional_combo]
                for values in itertools.product(*optional_values):
                    new_perm = base_perm.copy()
                    new_perm.update(dict(zip(optional_combo, values)))
                    yield new_perm

In [ ]:
# Generate function invocations with realistic parameter permutations


def generate_function_invocations(function_list: List[Dict]) -> List[Dict]:
    """
    Generate all possible function invocations from a list of functions.
    """
    all_invocations = []

    for function in function_list:
        func_name = function["function"]["name"]
        params = function["function"]["parameters"]

        # Skip functions with no parameters
        if not params.get("properties"):
            all_invocations.append({
                "name": func_name,
                "arguments": {}
            })
            continue

        # Generate all parameter permutations
        for arguments in generate_all_permutations(params):
            invocation = {
                "name": func_name,
                "arguments": arguments
            }
            all_invocations.append(invocation)

    return all_invocations

# Generate all invocations
invocations = generate_function_invocations(function_list)

In [ ]:
# Initialize the Gretel client


gretel = Gretel(api_key="prompt", validate=True, cache="yes")
tabular = gretel.factories.initialize_navigator_api("tabular")

In [ ]:
# Define a template for creating prompts for Gretel Navigator API
FILL_PARAMETERS_TEMPLATE = dedent("""
    Generate realistic values for the following parameters of the {function_name} function.
    Function description: {description}

    Columns to generate:
    {columns}

    Notes:
    - Values should be realistic and appropriate for computer control tasks.
    - All values must comply with the parameter constraints.
""").strip()


# Choose how many examples to synthesize for each invocation permutation
NUM_RECORDS_PER_INVOCATION = 2


def create_navigator_prompt(invocation: Dict[str, Any], function_def: Dict[str, Any]) -> str:
    """
    Generate a formatted prompt to guide Gretel Navigator in filling placeholder parameter values.

    Args:
        invocation: Dictionary containing the function name and arguments.
        function_def: Full function definition, including parameter metadata.

    Returns:
        A formatted prompt string or None if no placeholders need filling.
    """
    function_name = invocation["name"]
    arguments = invocation["arguments"]
    func_params = function_def["function"]["parameters"]

    # Collect parameter details for placeholders
    columns = []
    for param_name, value in arguments.items():
        if value in ["fill_in_int", "fill_in_string", "fill_in_object"]:
            param_info = func_params["properties"][param_name]
            param_desc = param_info.get("description", "")

            # Create column specifications based on the type
            if value == "fill_in_int":
                col_spec = f"- {param_name}: integer value"
                col_spec += f" (minimum: {param_info.get('minimum', 'N/A')})" if "minimum" in param_info else ""
                col_spec += f" (maximum: {param_info.get('maximum', 'N/A')})" if "maximum" in param_info else ""
            elif value == "fill_in_string":
                col_spec = f"- {param_name}: text value"
            elif value == "fill_in_object":
                col_spec = f"- {param_name}: JSON object"

            # Append description if available
            if param_desc:
                col_spec += f"\n  Description: {param_desc}"
            columns.append(col_spec)

    # Skip prompt creation if no placeholders
    if not columns:
        return None

    # Format the final prompt
    return FILL_PARAMETERS_TEMPLATE.format(
        function_name=function_name,
        description=function_def["function"].get("description", "No description available"),
        columns="\n".join(columns)
    )


def is_valid_invocation(name: str, args: Dict[str, Any]) -> bool:
    """
    Validate specific function arguments based on constraints.

    Args:
        name: The name of the function being invoked.
        args: The arguments provided for the function.

    Returns:
        Boolean indicating whether the invocation is valid.
    """
    return (
        # Control camera requires duration if mode is video
        (name != "control_camera" or args.get("mode") != "video" or
         1 <= args.get("duration", 0) <= 3600) and

        # Custom landing requires coordinates
        (name != "land_drone" or args.get("location") != "custom" or
         "coordinates" in args) and

        # Rainbow LED pattern should not specify a color
        (name != "configure_led_display" or args.get("pattern") != "rainbow" or
         "color" not in args)
    )


def fill_invocations(
    invocations: List[Dict],
    function_list: List[Dict],
    tabular,
    target_count: int = 50,
    num_records: int = 2
) -> List[Dict]:
    """
    Fill placeholders in function invocations with realistic values using Gretel Navigator.

    Args:
        invocations: List of function invocations with placeholder arguments.
        function_list: List of function definitions.
        tabular: Gretel Navigator tabular API instance.
        target_count: Number of filled invocations to generate.

    Returns:
        A list of filled invocations with realistic parameter values.
    """
    filled_invocations = []
    function_map = {f["function"]["name"]: f for f in function_list}
    attempts = 0

    while len(filled_invocations) < target_count and attempts < target_count * 2:
        for invocation in invocations:
            # Stop if target count is reached
            if len(filled_invocations) >= target_count:
                break

            # Get the function definition and create a prompt
            function_def = function_map[invocation["name"]]
            prompt = create_navigator_prompt(invocation, function_def)

            # Skip if no prompt is needed (e.g., no placeholders)
            if not prompt:
                if is_valid_invocation(invocation["name"], invocation["arguments"]):
                    filled_invocations.append(invocation)
                continue

            try:
                # Generate realistic values using the tabular API
                print(f"Synthesizing invocation data for {invocation['name']}")
                df = tabular.generate(prompt, num_records=2)

                # Update invocations with generated values
                for _, row in df.iterrows():
                    filled_inv = invocation.copy()
                    filled_args = filled_inv["arguments"].copy()

                    for col in df.columns:
                        if col in filled_args and filled_args[col] in ["fill_in_int", "fill_in_string", "fill_in_object"]:
                            value = row[col]
                            if filled_args[col] == "fill_in_int":
                                value = int(value)
                            elif filled_args[col] == "fill_in_object":
                                value = json.loads(value) if isinstance(value, str) else value
                            filled_args[col] = value

                    filled_inv["arguments"] = filled_args
                    if is_valid_invocation(filled_inv["name"], filled_args):
                        filled_invocations.append(filled_inv)

            except Exception as e:
                print(f"Error processing invocation {invocation['name']}: {e}")

        attempts += 1

    return filled_invocations[:target_count]


# Generate function invocations
invocations = generate_function_invocations(function_list)

# Fill placeholders with synthetic values using the tabular instance
filled_invocations = fill_invocations(invocations, function_list, tabular)

# Print filled invocations for debugging
for inv in filled_invocations:
    print(f"\n{inv['name']}:", json.dumps(inv['arguments'], indent=NUM_RECORDS_PER_INVOCATION))



In [ ]:
# Synthesize natural language commands that correspond to the invocations

def create_commands_prompt(df: pd.DataFrame) -> str:
    """
    Create a prompt to generate natural language commands for computer functions.

    Args:
        df: DataFrame containing computer functions and their parameters.

    Returns:
        A formatted prompt string to guide Gretel Navigator.
    """
    return dedent("""
    Add a new column called 'user_command' to the provided table containing natural, conversational commands
    that would result in calling these computer functions with these specific parameters.

    The commands should:
    - Be phrased naturally as a human would speak to a computer.
    - Vary in structure and wording (don't be repetitive).
    - Include the specific parameter values in a natural way.
    - Avoid technical jargon when possible.

    Example good commands:
    - "Open the file located at /home/user/documents/report.pdf"
    - "Search the web for 'best Python libraries for data analysis'"
    - "Run the command 'ls -la' in the terminal"
    """).strip()


def generate_commands(filled_invocations: List[Dict], tabular) -> pd.DataFrame:
    """
    Generate natural language commands based on filled invocations.

    Args:
        filled_invocations: List of filled function invocations.
        tabular: Gretel Navigator tabular API instance.

    Returns:
        A DataFrame with natural language commands added.
    """
    # Convert invocations into a DataFrame
    df = pd.DataFrame([{
        'function': inv['name'],
        'parameters': json.dumps(inv['arguments'], indent=2)  # Store parameters as a JSON string
    } for inv in filled_invocations])

    # Create the prompt for natural language command generation
    prompt = create_commands_prompt(df)

    # Generate conversational commands using Gretel Navigator
    df_with_commands = tabular.edit(prompt, seed_data=df)
    return df_with_commands


# Initialize Gretel client and Tabular API
gretel = Gretel(api_key="prompt", validate=True, cache="yes")
tabular = gretel.factories.initialize_navigator_api("tabular")

# Generate natural language commands
df_commands = generate_commands(filled_invocations, tabular)


In [ ]:
# Visualization of Generated Computer Commands

def display_results(df: pd.DataFrame, header_color: str = "blue", value_color: str = "black") -> None:
    """
    Display function commands, including rejection commands, with improved readability for white backgrounds.

    Args:
        df: DataFrame containing the columns 'function', 'parameters', and 'user_command'.
        header_color: Color for table headers (default: "blue").
        value_color: Color for table values (default: "black").
    """
    console = Console()

    # Create main table for displaying commands
    table = Table(
        title="Generated Commands",
        show_header=True,
        header_style=f"bold {header_color}"
    )
    table.add_column("Function", style=value_color)
    table.add_column("Parameters", style=value_color)
    table.add_column("User Command", style="dark_orange", overflow="fold")

    for _, row in df.iterrows():
        # Format parameters as pretty JSON
        params = json.dumps(json.loads(row['parameters']), indent=2)

        table.add_row(
            row['function'],
            Syntax(params, "json", theme="vim", background_color="default"),
            Text(row['user_command'], overflow="fold", style=value_color)
        )

    # Display the main table
    console.print("\n")
    console.print(Panel(table, title="Commands Table", expand=False))
    console.print("\n")

    # Create and display the summary table
    summary = Table(show_header=False, show_edge=False)
    summary.add_column(style=f"bold {header_color}")  # Header column
    summary.add_column(style=value_color)  # Value column
    summary.add_row("Total Commands:", str(len(df)))
    summary.add_row("Unique Functions:", str(df['function'].nunique()))

    console.print(Panel(summary, title="Summary", style=f"bold {header_color}"))

# Display results with adjusted header and value colors
display_results(df_commands, header_color="blue", value_color="black")



In [ ]:
# Generating Rejection Commands
# Rejection commands are critical for training AI systems to handle unsupported or ambiguous requests.

# By exposing the model to "realistic but unsupported" commands, we help it learn to reject requests
# that cannot be fulfilled, improving safety, robustness, and user experience.

REJECT_PROMPT = dedent("""
    Generate realistic-sounding computer commands that are almost feasible and are related to computer use,
    but that are not supported given the computer functions specified below.

    Here is a list of functions that the computer system supports:
    ```
    {function_list}
    ```

    The commands should:
    - Be phrased naturally as a human would speak to a computer assistant
    - Vary in structure and wording (don't be repetitive)
    - Include the specific parameter values in a natural way
    - Avoid technical jargon when possible

    The output should include:
    - A "user_request" column with each unsupported command
""").strip()

def generate_reject_data(tabular, function_list: List[Dict], num_records: int = 20) -> pd.DataFrame:
    """
    Generate rejection commands for unsupported computer requests.

    Args:
        tabular: Gretel Navigator tabular API instance.
        function_list: List of supported functions to reference in the prompt.
        num_records: Number of rejection commands to generate.

    Returns:
        A DataFrame containing rejection commands with standardized columns.
    """
    # Generate diverse impossible requests using Gretel Navigator
    df_reject = tabular.generate(REJECT_PROMPT.format(function_list=function_list), num_records=num_records)

    # Add function name and empty arguments for consistency
    df_reject['function'] = 'reject_request'
    df_reject['parameters'] = '{}'  # Empty JSON object for parameters
    df_reject = df_reject.rename(columns={'user_request': 'user_command'})

    # Ensure consistent column order
    return df_reject[['function', 'parameters', 'user_command']]

# Generate rejection data
df_reject = generate_reject_data(tabular, function_list)

# Combine rejection commands with valid commands
df_final = pd.concat([df_commands, df_reject], ignore_index=True)

# Display rejection commands
display_results(df_reject)


In [ ]:
# Analyzing Function Coverage
# This function analyzes how well each function is covered in the generated data.

# It provides insights into the number of examples for each function and the diversity of parameter values.
# Coverage analysis is critical to ensure that the dataset is comprehensive and balanced.



def analyze_coverage(df: pd.DataFrame, function_list: List[Dict]) -> None:
    """
    Analyze the coverage of generated examples for each function.

    Args:
        df: DataFrame containing generated examples, with columns 'function' and 'parameters'.
        function_list: List of function definitions, including parameter metadata.

    Displays:
        A Rich console table with:
        - Count of examples per function.
        - Diversity of parameter values for each function.
        - Summary of total examples, valid commands, and rejection cases.
    """
    console = Console()

    # Create main table for function coverage
    func_table = Table(title="Function Coverage Analysis", show_header=True, header_style="bold magenta")
    func_table.add_column("Function", style="blue")
    func_table.add_column("Count", justify="right", style="green")
    func_table.add_column("Parameters Coverage", style="yellow")

    # Map functions by name for quick lookup
    function_map = {f["function"]["name"]: f["function"] for f in function_list}

    # Group by function name and analyze coverage
    for func_name, group in df.groupby("function"):
        param_coverage = []
        if func_name != "reject_request":  # Skip reject cases for parameter coverage
            params_dict = function_map[func_name]["parameters"].get("properties", {})

            # Analyze parameter values
            for param_name in params_dict:
                values = []
                for _, row in group.iterrows():
                    args = json.loads(row["parameters"])
                    if param_name in args:
                        values.append(str(args[param_name]))

                if values:
                    value_counts = Counter(values)
                    unique_count = len(value_counts)
                    param_coverage.append(f"{param_name}: {unique_count} unique values")

        # Add function coverage data to the table
        func_table.add_row(
            func_name,
            str(len(group)),
            "\n".join(param_coverage) if param_coverage else "No parameters"
        )

    # Add summary statistics
    total_examples = len(df)
    reject_count = len(df[df["function"] == "reject_request"])
    valid_count = total_examples - reject_count

    # Create a summary table
    summary_table = Table.grid()
    summary_table.add_column(style="bold blue")  # Header column
    summary_table.add_column(style="green")      # Value column
    summary_table.add_row("Total Examples:", str(total_examples))
    summary_table.add_row("Valid Commands:", str(valid_count))
    summary_table.add_row("Reject Cases:", str(reject_count))

    # Display the coverage and summary tables
    console.print("\n")
    console.print(Panel(func_table, title="Function Coverage Analysis", expand=False))
    console.print(Panel(summary_table, title="Summary", expand=False))

# Analyze coverage for the combined dataset
analyze_coverage(df_final, function_list)


# 🛠 Next Steps: Applying your Synthetic Tool-Use Dataset

The synthesized dataset is designed to enhance AI systems in various tool-use scenarios, including fine-tuning and Retrieval-Augmented Generation (RAG) workflows. Here's how you can leverage it:



## 1️⃣ **Adapt for Your Use Case**
This notebook can be easily customized for any tool-use scenario, such as:
- **Home Automation:** Commanding smart devices with natural language inputs.
- **Robotics or Drones:** Generating datasets for navigation, task execution, or safety scenarios.
- **Custom Tools or APIs:** Training AI to interact with unique workflows, such as business software, shell scripting, or GUI automation.

By adapting the workflow to your tools and parameters, you can generate realistic datasets in a fraction of the time it would take to do it manually.


## 2️⃣ **Fine-Tuning a Model**
Use the dataset to fine-tune an AI model, enabling it to:
- **Specialize in your tools and commands:** Tailor the model’s behavior to your specific tool-use scenario.
- **Improve generalization:** Ensure the model reliably interprets user requests and executes corresponding functions.
- **Enhance rejection capabilities:** Teach the model to safely decline ambiguous or unsupported commands.

### Advantages of Fine-Tuning:
- Long-term memory for domain-specific tools and commands.
- High precision and reliability in executing defined functions.
- Ability to adapt to evolving tool requirements by retraining with updated datasets.

## 3️⃣ **In-Context Learning (RAG-Like Workflows)**
Leverage the dataset in RAG workflows or few-shot prompting to enable dynamic decision-making. For example:
- Use natural language commands to trigger tool use based on a few in-context examples.
- Incorporate rejection scenarios to refine model safety without the need for full retraining.

### Advantages of In-Context Learning:
- Flexibility to test new tools or commands without retraining.
- Quick prototyping and iteration with minimal data preparation.
- Enhanced interpretability by surfacing decision-making steps in real-time.


## 4️⃣ **Extend the Dataset**
Expand the dataset to cover:
- Additional tools or APIs specific to your domain.
- Complex workflows, such as multi-tool scenarios or cross-application interactions.
- Edge cases, including nuanced rejection scenarios, to improve robustness.


## 5️⃣ **Share Your Feedback**
Try this workflow on your own use case and let us know how it performs! Your feedback helps refine the approach and unlock new possibilities for tool-use AI systems.